# View npz data
This notebooks shows you how to visualize the data in npz files

In [2]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
print(sys.executable)
import os
if os.path.isfile("../models/brepnet.py"):
    os.chdir("../")
import tempfile
import numpy as np
    
import utils.data_utils as data_utils

# Imports from occwl
from occwl.io import load_step
from occwl.jupyter_viewer import JupyterViewer
from occwl.entity_mapper import EntityMapper


# Imports from BRepNet
from pipeline.extract_brepnet_data_from_step import extract_brepnet_data_from_step
import utils.scale_utils as scale_utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
c:\Users\dnkmn\anaconda3\envs\brepnet\python.exe


In [3]:
# Here is the path to some example step files for us to convert
step_folder = Path("./example_files/step_examples")
temp_folder = Path(tempfile.gettempdir())
working = temp_folder / "brepnet_test_working_dir"
if not working.exists():
    working.mkdir()

In [4]:
# Here we generate npz data from step files
extract_brepnet_data_from_step(step_folder, working)

100%|██████████| 25/25 [03:51<00:00,  9.28s/it]

Completed pipeline/extract_feature_data_from_step.py


In [5]:
all_npz_files = [ f for f in working.glob("*.npz") ]
npz_files = []
step_files = []
for npz_file in all_npz_files:
    step_file = step_folder / (npz_file.stem + ".stp")
    if step_file.exists():
        npz_files.append(npz_file)
        step_files.append(step_file)
    
print(f"Number of step and npz files {len(npz_files)}")

file_index = 4
npz_file = npz_files[file_index]
step_file = step_folder / (npz_file.stem + ".stp")
print("Viewing file") 
print(f"npz data {npz_file}")
print(f"STEP data {step_file}")

# Load the solid
solids = load_step(step_file)
print(f"Loaded {len(solids)} solids")
solid = solids[0]

# Scale to [-1,1]^3 box
solid = scale_utils.scale_solid_to_unit_box(solid)

# Load the npz data
data = data_utils.load_npz_data(npz_file)

Number of step and npz files 25
Viewing file
npz data C:\Users\dnkmn\AppData\Local\Temp\brepnet_test_working_dir\134103_d89213ee_0.npz
STEP data example_files\step_examples\134103_d89213ee_0.stp
Loaded 1 solids


In [6]:
viewer = JupyterViewer()
for face in solid.faces():
    viewer.display(face, render_edges=True)
viewer.show()

In [7]:
if len(viewer.selected_faces()) <= 0:
    print("Please select some faces to view this demo")
else:
    entity_mapper = EntityMapper(solid)
    selected_face_indices = viewer.selected_face_indices(entity_mapper)
    print(f"The faces you selected were {selected_face_indices}")

    # Get the point grids from the data
    face_grids = data["face_point_grids"]
    print(f"Shape of face grids {face_grids.shape}")
    selected_face_grids = face_grids[selected_face_indices]
    print(f"Shape of selected face grids {selected_face_grids.shape}")
    selected_face_grid_points = selected_face_grids[:,:3,:,:]
    points = np.transpose(np.reshape(selected_face_grid_points, (3, -1)))
    viewer.display_points(points)
    selected_face_grid_normals  = selected_face_grids[:,3:6,:,:]
    normals = np.transpose(np.reshape(selected_face_grid_normals, (3, -1)))
    viewer.display_unit_vectors(points, normals)

The faces you selected were [ 9 47  0  1 48  2]
Shape of face grids (166, 7, 10, 10)
Shape of selected face grids (6, 7, 10, 10)


# View edge reverse flag as color


In [8]:
edge_color_viewer = JupyterViewer()
edge_color_viewer.display(solid, render_edges=False)
for edge in solid.edges():
    if edge.reversed():
        color = "red"
    else:
        color = "blue"
    edge_color_viewer.display(edge, edge_color=color)
edge_color_viewer.show() 

# View edge reverse flag as direction

In [9]:
def display_dir_across_edge(viewer, coedge_grid):
    points = np.transpose(coedge_grid[:3, :])
    tangents = np.transpose(coedge_grid[3:6, :])
    left_normals = np.transpose(coedge_grid[6:9, :])
    across_edge_dir = np.cross(tangents, left_normals)
    viewer.display_points(points)
    viewer.display_unit_vectors(points, across_edge_dir, line_color="blue")

    
edge_dir_viewer = JupyterViewer()
edge_dir_viewer.display(solid, render_edges=True)
entity_mapper = EntityMapper(solid)
coedge_grids = data["coedge_point_grids"]
for edge in solid.edges():
    if edge.reversed():
        coedge = edge.reversed_edge()
    else:
        coedge = edge
    coedge_index = entity_mapper.oriented_edge_index(coedge)
    coedge_grid = coedge_grids[coedge_index]
    display_dir_across_edge(edge_dir_viewer, coedge_grid)
    
edge_dir_viewer.show()

# View the coedge grids

In [ ]:
coedge_viewer = JupyterViewer()
for face in solid.faces():
    coedge_viewer.display(face, render_edges=True)
coedge_viewer.show()

For this demo you need to select two adjacent faces and then we can visualize the coedge grid for the common edge.  The coedge is on the side of the first face.

In [ ]:
def display_coedge_grid(viewer, grid):
    points = np.transpose(grid[:3, :])
    tangents = np.transpose(grid[3:6, :])
    left_normals = np.transpose(grid[6:9, :])
    right_normals = np.transpose(grid[9:12, :])
    viewer.display_points(points)
    viewer.display_unit_vectors(points, tangents, line_color="red")
    viewer.display_unit_vectors(points, left_normals, line_color="blue")
    viewer.display_unit_vectors(points, right_normals, line_color="green")
    
def find_selected_coedge(viewer, entity_mapper):
    selected_faces = viewer.selected_faces()
    for index, selected_face in enumerate(selected_faces):
        selected_face_index = entity_mapper.face_index(selected_face)
        print(f"Face {index}: {selected_face_index}")
        
    selected_faces_set = set(selected_faces)
       
    # Find the coedge to the left of the face 0 and right of face 1
    target_coedge = None
    for coedge in solid.edges():
        ffe = set(solid.faces_from_edge(coedge))
        if ffe == selected_faces_set:
            print("Found edges which shares faces")
            mate = coedge.reversed_edge()
            if len(selected_faces) == 2:
                if selected_faces[0].is_left_of(coedge) and selected_faces[1].is_left_of(mate):
                    target_coedge = coedge
                elif selected_faces[0].is_left_of(mate) and selected_faces[1].is_left_of(coedge):
                    target_coedge = mate
            elif len(selected_faces) == 1:
                assert False, "Need to implement"
    return target_coedge
    
num_selected_faces = len(coedge_viewer.selected_faces())
print(f"Found {num_selected_faces} selected faces")
if num_selected_faces > 0:
    entity_mapper = EntityMapper(solid)
    target_coedge = find_selected_coedge(coedge_viewer, entity_mapper)
            
    if target_coedge is not None:
        coedge_index = entity_mapper.oriented_edge_index(target_coedge)
        print(f"Coedge index {coedge_index}")
        coedge_grids = data["coedge_point_grids"]
        selected_coedge_grids = coedge_grids[coedge_index]
        display_coedge_grid(coedge_viewer, selected_coedge_grids)
    else:
        print("Found no coedge")
else:
    print("Please select one face and one edge")

Found 0 selected faces
Please select one face and one edge


# View the coedge LCS matrix

In [ ]:
lcs_viewer = JupyterViewer()
for face in solid.faces():
    lcs_viewer.display(face, render_edges=True)
lcs_viewer.show()

In [ ]:
def display_coedge_lcs(viewer, selected_coedge_lcs):
    origin = np.expand_dims(selected_coedge_lcs[:, 3][:3], axis=0)
    u_vec = np.expand_dims(selected_coedge_lcs[:, 0][:3], axis=0)
    v_vec = np.expand_dims(selected_coedge_lcs[:, 1][:3], axis=0)
    w_vec = np.expand_dims(selected_coedge_lcs[:, 2][:3], axis=0)
    length = 0.02
    u_end = origin + u_vec*length
    v_end = origin + v_vec*length
    w_end = origin + w_vec*length
    viewer.display_points(origin)
    viewer.display_lines(origin, u_end, line_width=2, line_color="red")
    viewer.display_lines(origin, v_end, line_width=2, line_color="green")
    viewer.display_lines(origin, w_end, line_width=2, line_color="blue")

num_selected_faces = len(lcs_viewer.selected_faces())
print(f"Found {num_selected_faces} selected faces")
if num_selected_faces > 0:
    entity_mapper = EntityMapper(solid)
    target_coedge = find_selected_coedge(lcs_viewer, entity_mapper)
            
    if target_coedge is not None:
        coedge_index = entity_mapper.oriented_edge_index(target_coedge)
        print(f"Coedge index {coedge_index}")
        coedge_lcs = data["coedge_lcs"]
        selected_coedge_lcs = coedge_lcs[coedge_index]
        display_coedge_lcs(lcs_viewer, selected_coedge_lcs)
    else:
        print("Found no coedge")
else:
    print("Please select one face and one edge")

Found 0 selected faces
Please select one face and one edge


# View edge grids which UV-Net sees

In [ ]:
uv_net_edge_viewer = JupyterViewer()
uv_net_edge_viewer.display(solid, render_edges=True)
entity_mapper = EntityMapper(solid)
coedge_grids = data["coedge_point_grids"]
for edge in solid.edges():
    coedge_index = entity_mapper.oriented_edge_index(edge)
    coedge_grid = coedge_grids[coedge_index]
    display_coedge_grid(uv_net_edge_viewer, coedge_grid)
uv_net_edge_viewer.show()

KeyboardInterrupt: 

# View edge tangent directions

In [ ]:
def display_edge_tangents(viewer, grid):
    points = np.transpose(grid[:3, [0, 2, 4, 6, 8]])
    
    view_dir = np.array([1,1,-1])
    
    # Find a good length for the arrows
    mins = np.min(points, axis=0)
    maxs = np.max(points, axis=0)
    diag = maxs - mins
    longest = np.max(diag)
    arrow_length = longest/8   
    arrow_head = arrow_length/4
    
    points += view_dir*arrow_head
    
    tangents = np.transpose(grid[3:6, [0, 2, 4, 6, 8]])
    arrow_ends = points + arrow_length*tangents
    arrow_dir_cross = view_dir/np.linalg.norm(view_dir)
    arrow_offset1 = np.cross(tangents, arrow_dir_cross)
    arrow_offset2 = -arrow_offset1
    arrow1 = arrow_ends - (tangents+arrow_offset1)*arrow_head
    arrow2 = arrow_ends - (tangents+arrow_offset2)*arrow_head
    line_width = 2
    viewer.display_lines(points, arrow_ends, line_color="blue", line_width=line_width)
    viewer.display_lines(arrow1, arrow_ends, line_color="blue", line_width=line_width)
    viewer.display_lines(arrow2, arrow_ends, line_color="blue", line_width=line_width)
    
edge_tangent_viewer = JupyterViewer()
edge_tangent_viewer.display(solid, render_edges=True)
entity_mapper = EntityMapper(solid)
coedge_grids = data["coedge_point_grids"]
for edge in solid.edges():
    if edge.reversed():
        # If the edge had the reverse flag set then the code in 
        # the EdgeDataExtractor already reversed the tangent directions.
        # To get the same tangent directions as the edges 3d curve
        # we need to get the coedge grid from the mate
        coedge = edge.reversed_edge()
    else:
        coedge = edge
    coedge_index = entity_mapper.oriented_edge_index(coedge)
    coedge_grid = coedge_grids[coedge_index]
    display_edge_tangents(edge_tangent_viewer, coedge_grid)
edge_tangent_viewer.show()